In [1]:
import numpy as np
from pystrict import strict
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy import constants
from PyMPDATA import (Solver, Stepper, Options, ScalarField, VectorField,
                      ExtrapolatedBoundaryCondition, ConstantBoundaryCondition)
from matplotlib import pyplot



In [2]:
nrs=(1,)


def repeat_in_r(vec, nr):
    return np.repeat(vec.reshape(-1,1), nr, axis=1).squeeze()

In [3]:
class arakawa_c:    
    def z_scalar_coord(grid):
        zZ = np.linspace(1/2, grid[0]-1/2, grid[0])
        return zZ
    
    def z_vector_coord(grid):
        zZ = np.linspace(0, grid[0], grid[0]+1)
        return zZ

In [4]:
class MPDATA:
    def __init__(self, nz, dt, qv_of_zZ_at_t0, g_factor_of_zZ, mpdata_settings, nr):
        self.t = 0
        self.dt = dt
        self.fields = ('qv', 'ql')

        options = Options(
            n_iters=mpdata_settings['n_iters'],
            infinite_gauge=mpdata_settings['iga'],
            flux_corrected_transport=mpdata_settings['fct'],
            third_order_terms=mpdata_settings['tot']
        )
      
        
        self._solvers = {}
        for k in self.fields:
            grid = (nz, nr) if nr>1 and k =='ql' else (nz,)
            stepper = Stepper(options=options, grid=grid, non_unit_g_factor=True)

            data = g_factor_of_zZ(arakawa_c.z_scalar_coord(grid)) 
            if nr > 1 and k == 'ql':
                data = repeat_in_r(data, nr)
            g_factor = ScalarField(
                data=data,
                halo=options.n_halo,
                boundary_conditions=(ExtrapolatedBoundaryCondition(),)*len(data.shape)
            )
            
            if nr == 1 or k == 'qv':
                data = (np.zeros(nz+1),)
            else:
                data = (np.zeros((nz+1, nr)), np.zeros((nz, nr+1)))
            advector = VectorField(
                data=data, 
                halo=options.n_halo,
                boundary_conditions=(ExtrapolatedBoundaryCondition(),)*len(data)
                )
            
            if k == 'qv':
                data = qv_of_zZ_at_t0(arakawa_c.z_scalar_coord(grid))
                value = data[0]
                bcs = (ConstantBoundaryCondition(value),)
            else:
                data = np.zeros(grid)
                value = 0
                if nr == 1:
                    bcs = (ConstantBoundaryCondition(value),)*len(data.shape)
                else:
                    bcs = (ConstantBoundaryCondition(value), ConstantBoundaryCondition(1))
            advectee = ScalarField(
                data=data,
                halo=options.n_halo,
                boundary_conditions=bcs
            )
            self._solvers[k] = Solver(stepper=stepper, advectee=advectee, advector=advector, g_factor=g_factor)
            
    def __getitem__(self, k):
        return self._solvers[k]


In [5]:
def convert_to(value, unit):
    value /= unit

class si:
    kg = 1
    m = 1
    s = 1

    metres = 1
    second = 1
    um = 1e-6
    hPa = 100
    micrometre = 1e-6
    minutes = 60
    km = 1000
    dimensionless = 1
    kelvin = 1
    mg = 1e-6
    
    
class const:
    Mv = 0.018015
    Md = 0.028970
    eps = Mv / Md
    g = constants.g
    R_str = constants.R
    p1000 = 1000 * si.hPa
    Rd = 287.0027
    Rv = R_str / Mv
    c_pd = 1005
    c_pv = 1850
    lv = 2.5e6
    rho_l = 1e3 * si.kg / si.m**3
    T0 = constants.zero_Celsius    
    ARM_C1 = 6.1094 * si.hPa
    ARM_C2 = 17.625 * si.dimensionless
    ARM_C3 = 243.04 * si.kelvin

In [6]:
class Formulae:
    @staticmethod
    def rho_d(p, qv, theta_std):
        return p * (1 - 1 / (1 + const.eps / qv)) / (np.power(p / const.p1000, const.Rd / const.c_pd) * const.Rd * theta_std)
    
    @staticmethod
    def drho_dz(g, p, T, qv, lv, dql_dz=0):
        Rq = const.Rv / (1 / qv + 1) + const.Rd / (1 + qv)
        cp = const.c_pv / (1 / qv + 1) + const.c_pd / (1 + qv)
        rho = p / Rq / T
        return (g / T * rho * (Rq / cp - 1) - p * lv / cp / T**2 * dql_dz) / Rq
    
    # A14 in libcloudph++ 1.0 paper
    @staticmethod
    def T(rhod, thd):
        return thd * np.power(rhod * thd / const.p1000 * const.Rd, const.Rd / const.c_pd / (1 - const.Rd / const.c_pd))

    # A15 in libcloudph++ 1.0 paper
    @staticmethod
    def p(rhod, T, qv):
        return rhod * (1 + qv) * (const.Rv / (1 / qv + 1) + const.Rd / (1 + qv)) * T
    
    @staticmethod
    def th_dry(th_std, qv):
        return th_std * np.power(1 + qv / const.eps, const.Rd / const.c_pd)
    
    @staticmethod
    def pvs_Celsius(T):
        return const.ARM_C1 * np.exp((const.ARM_C2 * T) / (T + const.ARM_C3))

    @staticmethod
    def pv(p, qv):
        return p * qv / (qv + const.eps)

In [7]:
@strict
class Settings:
    def __init__(self, w_1: float, nr: int = 1):
        
        self.dz = 25*si.m
        self.dt = 1*si.s
        
        self.nr = nr
        self.ksi_1 = 100 * si.micrometre**2 / si.second # TODO #221: import from Olesik?
        

        self.z_max = 3000 * si.metres
        self.t_max = 15 * si.minutes
        
        self.mpdata_settings = {'n_iters': 3, 'iga': True, 'fct': True, 'tot': True}
        self.qv = interp1d((0, 740, 3260), (.015, .0138, .0024))  
        self._th = interp1d((0, 740, 3260), (297.9, 297.9, 312.66))

        # note: not in the paper, https://github.com/BShipway/KiD/blob/bad81aa6efa4b7e4743b6a1867382fc74c10a884/src/physconst.f90#L43
        p0 = 1000 * si.hPa  
        
        self.rhod0 = Formulae.rho_d(p0, self.qv(0), self._th(0))
        self.thd = lambda z: Formulae.th_dry(self._th(z), self.qv(z))

 
        # TEMP !!!
        self.mpdata_settings['n_iters'] = 1
        self.mpdata_settings['fct'] = False
        
        
        def drhod_dz(z, rhod):
            T = Formulae.T(rhod[0], self.thd(z))
            p = Formulae.p(rhod[0], T, self.qv(z))
            return Formulae.drho_dz(const.g, p, T, self.qv(z), const.lv)

        z_points = np.arange(0, self.z_max+self.dz/2, self.dz / 2)
        rhod_solution = solve_ivp(
            fun=drhod_dz,
            t_span=(0, self.z_max),
            y0=np.asarray((self.rhod0,)),
            t_eval=z_points
        )
        assert rhod_solution.success
        
        self.rhod = interp1d(z_points, rhod_solution.y[0])
    
        t_1 = 600 * si.s
        self.w = lambda t: w_1 * np.sin(np.pi * t/t_1) if t < t_1 else 0

        self.r_min = 5 * si.um
        self.r_max = 25 * si.um
        self.bin_boundaries, self.dr = bin_locations = np.linspace(
            self.r_min,
            self.r_max,
            self.nr+1, retstep=True
        )
        self.dr4 = self.bin_boundaries[1:]**4 - self.bin_boundaries[:-1]**4
        self.dr4 = self.dr4.reshape(1,-1).T
        
        self.z_vec = self.dz*arakawa_c.z_vector_coord((self.nz,))
    
    @property
    def nz(self):
        nz = self.z_max / self.dz
        assert nz == int(nz)
        return int(nz)
    
    @property
    def nt(self):
        nt = self.t_max / self.dt
        assert nt == int(nt)
        return int(nt)


In [8]:
def plot(var, mult, qlabel, fname, output):
    dt = output['t'][1] - output['t'][0]
    dz = output['z'][1] - output['z'][0]
    tgrid = np.concatenate(((output['t'][0] - dt / 2,), output['t'] + dt / 2))
    zgrid = np.concatenate(((output['z'][0] - dz / 2,), output['z'] + dz / 2))
    convert_to(zgrid, si.km)

    fig = pyplot.figure(constrained_layout=True)
    gs = fig.add_gridspec(25, 5)
    ax1 = fig.add_subplot(gs[:-1, 0:4])
    mesh = ax1.pcolormesh(tgrid, zgrid, output[var]*mult, cmap='twilight', rasterized=True)

    ax1.set_xlabel('time [s]')
    ax1.set_ylabel('z [km]')

    cbar = fig.colorbar(mesh, fraction=.05, location='top')
    cbar.set_label(qlabel)

    ax2 = fig.add_subplot(gs[:-1, 4:], sharey=ax1)
    ax2.set_xlabel(qlabel)
    #ax2.set_yticks([], minor=False)

    last_t = -1
    for i, t in enumerate(output['t']):
        x, z = output[var][:, i]*mult, output['z'].copy()
        convert_to(z, si.km)
        params = {'color': 'black'}
        for line_t, line_s in {3: ':', 6: '--', 9: '-', 12: '-.'}.items():
            if last_t < line_t * si.minutes <= t:
                print(f"plotting line at={t}")
                params['ls'] = line_s
                ax2.plot(x, z, **params)
                ax1.axvline(t, **params)
        last_t = t
    pyplot.show()
#    show_plot(filename=fname)

In [9]:
def label(w, nr):
    return f"nr={nr}_w={w}"

In [10]:
outputs = {}
for w in (2*si.m/si.s, 3*si.m/si.s):
    for nr in nrs:
        print("Simulating using nr={}:".format(nr))
        
        settings = Settings(w_1=w, nr=nr)

        mpdata = MPDATA(
            nr=nr,
            nz=settings.nz, 
            dt=settings.dt, 
            mpdata_settings = settings.mpdata_settings,
            qv_of_zZ_at_t0 = lambda zZ: settings.qv(zZ*settings.dz),
            g_factor_of_zZ = lambda zZ: settings.rhod(zZ*settings.dz) # TODO #221 spectral coord
        )

        output = {k: np.zeros((settings.nz, settings.nt+1)) for k in ('qv', 'S', 'ql')}
     
        assert 't' not in output and 'z' not in output
        output['t'] = np.linspace(0, settings.nt*settings.dt, settings.nt+1, endpoint=True)
        output['z'] = np.linspace(settings.dz/2, (settings.nz-1/2)*settings.dz, settings.nz, endpoint=True)
        output['qv'][:,0] = mpdata['qv'].advectee.get()
        output['ql'][:,0] = 0

        prof = {}
        prof['rhod'] = settings.rhod(output['z'])
        prof['T'] = Formulae.T(prof['rhod'], settings.thd(output['z']))
        prof['p'] = Formulae.p(prof['rhod'], prof['T'], output['qv'][:,0])
        prof['pvs'] = Formulae.pvs_Celsius(prof['T']-const.T0)

        Gvec = settings.rhod(settings.z_vec)
        Gscl = prof['rhod']

        for t in range(settings.nt):
            print("t=", t)
            C = settings.w((t+.5) * settings.dt) * settings.dt / settings.dz
            advector_0 = Gvec * C
            mpdata['qv'].advector.get_component(0)[:] = advector_0
            mpdata['qv'].advance(1)
            qv = mpdata['qv'].advectee.get()
            RH = Formulae.pv(prof['p'], qv) / prof['pvs']

            if nr==1:
                mpdata['ql'].advector.get_component(0)[:] = advector_0
            else:
                mpdata['ql'].advector.get_component(0)[:, :] = np.repeat(advector_0.reshape(1,-1).T, nr, axis=1)
                if np.amax(RH)>1:
                    GC = mpdata['ql'].advector.get_component(1)
                    print("pre", np.amin(GC), np.amax(GC))
                mpdata['ql'].advector.get_component(1)[:, :] = (
                    # scalar
                    settings.ksi_1 * settings.dt / settings.dr # m2/s * s / m
                    # column
                    * np.repeat((Gscl * (np.maximum(1, RH)-1)).reshape(1,-1).T, nr+1, axis=1) # kg/m3
                    # spectrum
                    / np.repeat(settings.bin_boundaries.reshape(1,-1), settings.nz, axis=0) # m-1       
                )
                if np.amax(mpdata['ql'].advector.get_component(1))>1:
                    assert False
                    
            mpdata['ql'].advance(1)

            if nr==1:
                ql = mpdata['ql'].advectee.get()


                # TODO #221: iterate
                # TODO #221: assert not np.logical_and(S<1, ql>0).any()
                dql = np.maximum(0, qv * (1 - 1/RH))
                ql += dql
            else:
                psi = mpdata['ql'].advectee.get()
                z = 10
                
                
#                 psi[z, -1] = 100 / si.mg / settings.dr
                
                ql = 4/3*np.pi*const.rho_l*np.sum(np.dot(psi, settings.dr4/4), axis=1)
                if t == -1:
                    pyplot.step(x=settings.bin_boundaries, y=np.concatenate(([0], psi[z,:])), where='pre')
                    pyplot.xlabel("radius [m]")
                    pyplot.ylabel("psi [1/m 1/kg]")
                    
                    # advection/conservation of n/V, but G_factor=rho_d(z)*1, so psi=n/V/rho
                    
                    print("ql=", ql)  # note: 4=l+1
                    assert False

                dql = ql - output['ql'][:,t]

            qv -= dql
                
            output['ql'][:,t+1] = ql    
            output['qv'][:,t+1] = qv
            output['S'][:,t+1] = RH - 1
            
        outputs[label(w, nr)] = output
        print("finished")

Simulating using nr=100:
t= 0
pre 0.0 0.0
t= 1
pre 0.0 0.9993098002449751
t= 2
pre 0.0 0.9996465444822655


AssertionError: 

In [11]:
for nr in nrs:
    print("Plotting results for nr {}:".format(nr))
    for w in (2, 3):
        key = label(w, nr)
        plot(var='qv', mult=1e3, qlabel='$q_v$ [g/kg]', fname=f'qv_{key}.pdf', output=outputs[f'{key}.0'])
        plot(var='ql', mult=1e3, qlabel='$ql$ [g/kg]', fname=f'ql_{w}.pdf', output=outputs[f'{key}.0'])
        plot(var='S', mult=1e2, qlabel='$S$ [%]', fname=f'S_{key}.pdf', output=outputs[f'{key}.0'])

Plotting results for nr 100:


KeyError: 'nr=100_w=2.0'